Test different models and best parameters  
Models: denoising ae, sparse ae, contractive ae 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
import keras.backend as K


import innvestigate
import innvestigate.utils as iutils

import numpy as np
import pandas as pd
import pickle

from sklearn import preprocessing

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


### Setup

Stop monitor

In [3]:
early_stopping_monitor = keras.callbacks.EarlyStopping(monitor='loss', patience=20, verbose=1)

Feature matrix

In [4]:
pklfile = '../../data/all_bands.pkl'
with open(pklfile, 'rb') as fi:
    data = pickle.load(fi)

Scaling

In [5]:
data = preprocessing.MinMaxScaler().fit_transform(np.abs(data))

In [6]:
data.shape

(42820, 1622)

### Models

Denoising AE

In [7]:
def denoisingAutoencoder(feat_mat, noise_factor=0.05, hidden_size=800, batch_size=128, epochs=2000):
    
    input_size = feat_mat.shape[1]
    feat_noisy = feat_mat + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=feat_mat.shape) 
    feat_noisy = np.clip(feat_noisy, 0., 1.)

    x = Input(shape=(input_size,))
    h = Dense(hidden_size, activation='relu')(x)
    r = Dense(input_size, activation='sigmoid')(h)

    ae = Model(inputs=x, outputs=r)
    ae.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

    history = ae.fit(feat_noisy, feat_noisy, batch_size=batch_size,epochs=epochs, callbacks = [early_stopping_monitor])
    encoder = Model(x,h)
    return ae, encoder

Sparse AE

In [8]:
def sparseAutoencoder(feat_mat, l=1e-5, hidden_size=800, batch_size=128, epochs=2000):
    
    input_size = feat_mat.shape[1]


    x = Input(shape=(input_size,))
    h = Dense(hidden_size, activation='relu', activity_regularizer=keras.regularizers.l1(l))(x)
    r = Dense(input_size, activation='sigmoid')(h)

    ae = Model(inputs=x, outputs=r)
    ae.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

    history = ae.fit(feat_mat, feat_mat, batch_size=batch_size,epochs=epochs, callbacks = [early_stopping_monitor])
    encoder = Model(x,h)
    return ae, encoder

Contractive AE

In [9]:
def contractiveAutoencoder(feat_mat, hidden_size=800, lam=1e-5, batch_size=128, epochs=2000):

    input_size = feat_mat.shape[1]
    x = Input(shape=(input_size,))
    h = Dense(hidden_size, activation='relu', name='encoded')(x)
    r = Dense(input_size, activation='sigmoid')(h)

    ae= Model(inputs=x, outputs=r)

    def contractive_loss(y_pred, y_true):
        mse = K.mean(K.square(y_true - y_pred), axis=1)

        W = K.variable(value=ae.get_layer('encoded').get_weights()[0])  # N x N_hidden
        W = K.transpose(W)  # N_hidden x N
        h = ae.get_layer('encoded').output
        dh = h * (1 - h)  # N_batch x N_hidden

        # N_batch x N_hidden * N_hidden x 1 = N_batch x 1
        contractive = lam * K.sum(dh**2 * K.sum(W**2, axis=1), axis=1)

        return mse + contractive


    ae.compile(optimizer='adam', loss=contractive_loss, metrics=['accuracy'])
    history = ae.fit(feat_mat, feat_mat, batch_size=batch_size,epochs=epochs, callbacks = [early_stopping_monitor])  
    encoder = Model(x, h)
    

    return ae, encoder

### DAE performance

Core size

In [ ]:
core_list = [512, 800, 1024, 1500, 2048]
models_dae_core = []

for core_size in core_list:
    dae, dae_encoder = denoisingAutoencoder(data, hidden_size=core_size)
    models_dae_core.append([dae, dae_encoder])








Epoch 1/2000
42820/42820 [==============================] - 3s 64us/step - loss: 0.0098 - acc: 0.0025
Epoch 2/2000
42820/42820 [==============================] - 1s 29us/step - loss: 0.0045 - acc: 0.0063
Epoch 3/2000
42820/42820 [==============================] - 1s 29us/step - loss: 0.0038 - acc: 0.0122
Epoch 4/2000
42820/42820 [==============================] - 1s 29us/step - loss: 0.0029 - acc: 0.0316
Epoch 5/2000
42820/42820 [==============================] - 1s 29us/step - loss: 0.0024 - acc: 0.0517
Epoch 6/2000
42820/42820 [==============================] - 1s 29us/step - loss: 0.0022 - acc: 0.0669
Epoch 7/2000
35968/42820 [========================>.....] - ETA: 0s - loss: 0.0021 - acc: 0.0793

In [1]:
pklfile = '../../data/models_dae_core.pkl'
with open(pklfile, 'wb') as fo:
    pickle.dump(models_dae_core, fo)

NameError: name 'pickle' is not defined

### Test

In [ ]:
ae, encoder = autoencoder(data, epochs=1)

In [8]:
ae, encoder = denoisingAutoencoder(data, epochs=2)







Epoch 1/2
42820/42820 [==============================] - 3s 64us/step - loss: 0.0091 - acc: 0.0020
Epoch 2/2
42820/42820 [==============================] - 1s 35us/step - loss: 0.0047 - acc: 0.0041


In [ ]:
ae, encoder = dae(data, epochs=1)

In [ ]:
ae, encoder = dae(data, epochs=2)

In [ ]:
ae, encoder = cae(data, epochs=1)

In [ ]:
ae, encoder = cae(data, epochs=2)

In [ ]:
sae, s_encoder = sae(data, epochs=20)

In [ ]:
cae, c_encoder = cae(data, epochs=1)

In [ ]:
cae, c_encoder = cae(data, epochs=2, lam=1e-6)

In [ ]:
[2**i for i in range(9,12)]

In [13]:
ae.history.history

{'loss': [0.009093473095486845, 0.004698725807315651],
 'acc': [0.002008407286401805, 0.003713218122372723]}